## Проект 6. Выбираем авто выгодно

## Задача

#### Построить модель определения цены автомобиля на вторичном рынке по основным характеристикам

## Проблема

#### Отсутствие истории продаж и актуальной информации о предложениях на рынке

## Основные инструменты

* #### [Авто.ру](https://auto.ru/)  
* #### [Библиотека scikit-learn](https://scikit-learn.org/stable/)  
* #### [Программная среда optuna](https://optuna.org/)  
* #### [Проэкт автоматизации браузеров Селениум](https://www.selenium.dev/) 
* #### [Пакет для анализа документов BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

In [2]:
import numpy as np
import pandas as pd
import sys

from sklearn import preprocessing
from sklearn.preprocessing import (
    LabelEncoder, PolynomialFeatures,
    StandardScaler
)

from sklearn.cluster import DBSCAN

from sklearn.model_selection import (
    RandomizedSearchCV, train_test_split,
    KFold
)

from sklearn.feature_selection import f_classif, mutual_info_classif

from sklearn.ensemble import (
    RandomForestRegressor, ExtraTreesRegressor,
    AdaBoostRegressor, GradientBoostingRegressor
)

from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression

from tqdm.notebook import tqdm

from catboost import CatBoostRegressor

import re
import pandas_profiling
import json
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import optuna

In [3]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [5]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

In [6]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [7]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [8]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

### Функция выделения комплектации

In [9]:
def complectation_cut(complectation):
    try:
        if 'available_options' in complectation:
            complectation_new = eval(complectation)['available_options']
            return complectation_new
    except:
        return None

### Функция удаления латиницы и символов

In [10]:
def cut_bodytype(bodytype):
    try:
        bodytype_new = re.split(r'[A-Z,a-z,0-9]', bodytype)[0].rstrip()
    except:
        bodytype_new = bodytype
        
    return str(bodytype_new).lower()

### Функция поиска расхода топлива

In [11]:
def engine_displacement_cut(engine):
    engine = str(engine)
    point_place = engine.find('.')
    if point_place != -1:
        engine_new_str = engine[point_place-1:point_place+2]
        engine_new_float = float(engine_new_str)
        return engine_new_float
    else:
        return None

### Функции расчёта врмемени владением авто

In [12]:
"""
Для работы с признаком из train
Функция подсчета времени владения автомобилем с времени приобретения до 09.09.2020.
Единица измерения - год.
"""
def time_ownership(ownership):
    try:
        year_of_buy = int(ownership.replace("'year': ",'').replace(" 'month': ",'')[1:5])
        month_of_buy = int(ownership.replace("'year': ",'').replace(" 'month': ",'')[6:-1])
    
        year = (2020 - year_of_buy)
        month = 12 - month_of_buy + 9
    
        if month>=12:
            year+=1
            month-=12
        
        time_ownership = year + month/12

        
        return time_ownership
    except:
        pass

In [13]:
"""
Для работы с признаком из test
Единица измерения - год.
"""
def time_ownership_test(ownership):
    try:
        if (('год' in ownership) or ('лет' in ownership)) and ('мес' in ownership):
            try:
                year = int(re.split(r' год', ownership)[0])
            except:
                year = int(re.split(r' лет', ownership)[0])
            month = int(re.split(r' и ', ownership)[1].split(' ')[0])

        elif (('год' in ownership) or ('лет' in ownership)) and not ('мес' in ownership):
            try:
                year = int(re.split(r' год', ownership)[0])
                month = 0
            except:
                year = int(re.split(r' лет', ownership)[0])
                month = 0
    
        elif not(('год' in ownership) or ('лет' in ownership)) and ('мес' in ownership):
            year = 0
            month = int(ownership.split(' ')[0])
    
        else:
            None
    
        time_ownership = year + month/12
    
        return time_ownership
    except:
        pass

## Функция построения группы боксплотов

In [14]:
def boxplot_out(df, columns):
    fig = plt.figure(figsize=[30, 8])
    count = 1
    for each in columns:
        ax = fig.add_subplot(1, 5, count)
        sns.boxplot(y=df[each], data=df[each], orient='v')
        ax.set_title(label=each, fontdict={'fontsize': 12})
        ax.set(ylabel='')
        plt.subplots_adjust(wspace=1)
        count += 1
    plt.show()

## Функция визуализации выбросов

In [15]:
def graph_out(df, columns):
    fig = plt.figure(figsize=[30, 4])
    count = 1
    for each in columns:
        ax = fig.add_subplot(1, len(columns), count)
        plt.title(each)

        perc25 = df[each].quantile(0.25)
        perc75 = df[each].quantile(0.75)
        IQR = perc75 - perc25

        df[each].loc[df[each].between(
            perc25 - 1.5*IQR, perc75 + 1.5*IQR)].hist(label='IQR')
        df[each].loc[(df[each] < perc25 - 1.5*IQR) |
                     (df[each] > perc75 + 1.5*IQR)].hist(label='выбросы')

        count += 1

        plt.legend()

## Функция статистики / удаления шума по z_score

In [16]:
def outliers(df, columns, function):
    for each in columns:
        mean = np.mean(df[each])
        std = np.std(df[each])
        moda = df[each].mode()
        median = df[each].median()
        z_score = [(x-mean)/std for x in df[each]]
        out_z = np.where(np.abs(z_score) > 3)[0]
        out_z_count = len(np.where(np.abs(z_score) > 3)[0])
        out_z_perc = round(100*out_z_count / df.shape[0], 2)
        if function == 'search':
            print(f'Количество выбросов у {each} по Z-score: {out_z_count}, {out_z_perc}% выборки.')
        
        if function == 'delete':
            for i in out_z:
                df[each][i] = median            

## Функция группы гистограмм

In [17]:
def hist_type(df, columns):
    fig = plt.figure(figsize=[70, 70])
    count = 1
    for each in columns:
        ax = fig.add_subplot(7, 2, count)
        sns.countplot(x=each, data=df,
                      order=df[each].value_counts().index)
        ax.tick_params(axis='x', labelsize=14, labelrotation=90)
        ax.set_title(label=each, fontdict={'fontsize': 16})
        count += 1

# Setup

In [18]:
VERSION    = 16
DIR_TRAIN  = '../input/parsing-all-moscow-auto-ru-09-09-2020/' # подключил к ноутбуку внешний датасет
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%

# Data

In [19]:
train = pd.read_csv(DIR_TRAIN+'all_auto_ru_09_09_2020.csv') # датасет для обучения модели
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

In [20]:
train.sample(5)

In [21]:
train.info()

In [22]:
test.head(5)

In [23]:
test.info()

## Фильтр на авто с пробегом

In [24]:
train[train['mileage']==0].shape

In [25]:
test[test['mileage']==0].shape

In [26]:
train[train['mileage']==0]['Владельцы'].unique()

В выборке 13588 автомобилей с нулевым пробегом и не имеющих ни одного владельца,  
то есть можно сделать предположение, что это новые автомобили.  
Это косвенно подтверждают комментарии, в которых описывается официальный диллер.
Эти строки противоречат поставленной задаче определения цены авто на вторичном рынке.   
Поэтому строки следует удалить.  
В результате уменьшится только обучающая выборка

In [27]:
train = train[train['mileage']>0].reset_index(drop=True)

## Data Preprocessing

In [28]:
train[(train['productionDate']==1989) & (train['brand'] == 'HONDA')]

Обнаружено авто с малым объём признаков,которые невозможно восстановить из описания. Данное авто судя по индексу из train, поэтому решено удалить этот пример

In [29]:
index_nan = train[(train['productionDate']==1989) & (train['brand'] == 'HONDA')].index[0]

In [30]:
train.iloc[index_nan]

In [31]:
train = train.drop(index=index_nan).reset_index(drop=True)

Для того, чтобы модели правильно обучились и работали, надо признаки train привести в соответствие с признаками test

### Приведение в соответствие признака body_type

In [32]:
train['bodyType'] = train['bodyType'].map(cut_bodytype)
test['bodyType'] = test['bodyType'].map(cut_bodytype)

In [33]:
train['bodyType'].unique()

In [34]:
test['bodyType'].unique()

In [35]:
set(test['bodyType'].unique()).difference(set(train['bodyType'].unique()))

In [36]:
train.rename(columns={'bodyType' : 'body_type' }, inplace=True)
test.rename(columns={'bodyType' : 'body_type' }, inplace=True)

Все виды признака из тестовой выборки присутствуют в учебной выборке

### Приведение в соответствие признака brand

In [37]:
train['brand'].unique()

In [38]:
test['brand'].unique()

In [39]:
set(test['brand'].unique()).difference(set(train['brand'].unique()))

Все бренды из тестовой выборки присутствуют в учебной выборке

### Приведение в соответствие признака color

In [40]:
train['color'].unique()

In [41]:
test['color'].unique()

In [42]:
color_dict = {'040001': 'чёрный', 'FAFBFB': 'белый', '97948F': 'серый', 'CACECB': 'серебристый', '0000CC': 'синий', '200204': 'коричневый',
              'EE1D19': 'красный',  '007F00': 'зелёный', 'C49648': 'бежевый', '22A0F8': 'голубой', '660099': 'пурпурный', 'DEA522': 'золотистый', 
              '4A2197': 'фиолетовый', 'FFD600': 'жёлтый', 'FF8649': 'оранжевый', 'FFC0CB': 'розовый'}

In [43]:
train['color'] = train['color'].replace(color_dict)

In [44]:
set(test['color'].unique()).difference(set(train['color'].unique()))

Все цвета из тестовой выборки присутствуют в учебной выборке

### Приведение в соответствие признака complectation

Данный признак - источник дополнительных опций

In [45]:
train['Комплектация'] = train['Комплектация'].map(complectation_cut)

In [46]:
test['complectation_dict'] = test['complectation_dict'].map(complectation_cut)

In [47]:
train.rename(columns={'Комплектация' : 'complectation_dict' }, inplace=True)

### Приведение в соответствие признака description

In [48]:
train['description'].value_counts().head(3)

In [49]:
test['description'].value_counts().head(3)

На данном этапе признак не нуждается в трансформации.

### Приведение в соответствие признака engineDisplacement

In [50]:
train['engineDisplacement'].unique()

In [51]:
test['engineDisplacement'].unique()

В train содержится смешанная информация, не всегда соответствующая объёму двигателя.

In [52]:
train['engineDisplacement'] = train['name'].map(engine_displacement_cut)

In [53]:
test['engineDisplacement'] = test['engineDisplacement'].apply(lambda x: re.split(r'[A-Z,a-z]', x)[0].rstrip())  ## Удаление единицы измерения

In [54]:
test['engineDisplacement'] = test['engineDisplacement'].apply(lambda x: 0 if x in [None, ''] else float(x)) ## Электромобилям присвоим нулевое значение

In [55]:
train.rename(columns={'engineDisplacement' : 'engine_displacement' }, inplace=True)
test.rename(columns={'engineDisplacement' : 'engine_displacement' }, inplace=True)

In [56]:
set(test['engine_displacement'].unique()).difference(set(train['engine_displacement'].unique()))

По данному признаку train содержит элементы test

### Приведение в соответствие признака enginePower

In [57]:
train['enginePower'].unique()

In [58]:
test['enginePower'].unique()

In [59]:
## Возврат числовой величины
test['enginePower'] = test['enginePower'].apply(lambda x: re.split(r'[A-Z,a-z]', x)[0].rstrip())
test['enginePower'] = test['enginePower'].apply(lambda x: int(x))

In [60]:
train.rename(columns={'enginePower' : 'engine_power' }, inplace=True)
test.rename(columns={'enginePower' : 'engine_power' }, inplace=True)

In [61]:
set(test['engine_power'].unique()).difference(set(train['engine_power'].unique()))

В train нет данных с мощностью 387 л.с. и 522 л.с.

### Приведение в соответствие признака equipment_dict

In [62]:
test['equipment_dict'].unique()

equipment_dict находится только в test и, возможно, дублирует complectation. поэтому этот признак можно удалить

In [63]:
test = test.drop(['equipment_dict'],axis=1)

### Приведение в соответствие признака fuelType

In [64]:
train['fuelType'].unique()

In [65]:
test['fuelType'].unique()

In [66]:
train.rename(columns={'fuelType' : 'fuel_type' }, inplace=True)
test.rename(columns={'fuelType' : 'fuel_type' }, inplace=True)

In [67]:
set(test['fuel_type'].unique()).difference(set(train['fuel_type'].unique()))

По данному признаку train содержит элементы test

### Приведение в соответствие признака image

В train данный признак отсутствует, поэтому без парсинга лучше удалить его и в test

In [68]:
test = test.drop(['image'],axis=1)

### Приведение в соответствие признака modelDate

In [69]:
train['modelDate'] = train['modelDate'].astype(int)

In [70]:
train.rename(columns={'modelDate' : 'model_date' }, inplace=True)
test.rename(columns={'modelDate' : 'model_date' }, inplace=True)

### Приведение в соответствие признака model_name

In [71]:
test.rename(columns={'model_name' : 'model'}, inplace=True)

In [72]:
set(test['model'].unique()).difference(set(train['model'].unique()))

В train представлены не все модели 

### Приведение в соответствие признака model_info

Признаки model и model_info дублируют друг друга

In [73]:
test = test.drop(['model_info'],axis=1)

### Приведение в соответствие признака name

Признак дублирует информацию fuel_type, engine_power, engine_displacement.  
Решено удалить данный признак

In [74]:
test = test.drop(['name'],axis=1)

In [75]:
train = train.drop(['name'],axis=1)

### Приведение в соответствие признака numberOfDoors

In [76]:
train['numberOfDoors'].value_counts()

In [77]:
test['numberOfDoors'].value_counts()

In [78]:
train['numberOfDoors'] = train['numberOfDoors'].astype(int)

In [79]:
train.rename(columns={'numberOfDoors' : 'number_of_doors'}, inplace=True)
test.rename(columns={'numberOfDoors' : 'number_of_doors'}, inplace=True)

### Приведение в соответствие признака parsing_unixtime

признак не информативен

In [80]:
test = test.drop(['parsing_unixtime'],axis=1)      

### Приведение в соответствие признака priceCurrency

In [81]:
test['priceCurrency'].value_counts()

Признак не информативен

In [82]:
test = test.drop(['priceCurrency'],axis=1)      

### Приведение в соответствие признака productionDate

In [83]:
train.rename(columns={'productionDate' : 'production_date' }, inplace=True)
test.rename(columns={'productionDate' : 'production_date' }, inplace=True)

### Приведение в соответствие признака sell_id

In [84]:
train['sell_id'] = 0

### Приведение в соответствие признака super_gen

Признак отсутсвует в трейне, поэтому не является полезным для обучения

In [85]:
test = test.drop(['super_gen'],axis=1) 

### Приведение в соответствие признака vehicleConfiguration

Признак сочитает информацию о body_type, vehicle_transmission, number_of_doors, engine_displacement

In [86]:
train = train.drop(['vehicleConfiguration'],axis=1)  
test = test.drop(['vehicleConfiguration'],axis=1)

### Приведение в соответствие признака vehicleTransmission

In [87]:
train['vehicleTransmission'].value_counts()

In [88]:
test['vehicleTransmission'].value_counts()

In [89]:
vehicle_transmission_dict = {'AUTOMATIC': 'автоматическая', 'MECHANICAL': 'механическая',
                             'ROBOT': 'роботизированная', 'VARIATOR': 'вариатор'}

In [90]:
train['vehicleTransmission'] = train['vehicleTransmission'].replace(vehicle_transmission_dict)

In [91]:
train.rename(columns={'vehicleTransmission' : 'vehicle_transmission' }, inplace=True)
test.rename(columns={'vehicleTransmission' : 'vehicle_transmission' }, inplace=True)

### Приведение в соответствие признака vendor

In [92]:
test['vendor'].unique()

In [93]:
test.groupby(['vendor','brand'])['body_type'].count()

Признак отсутствует в train. В train есть модели не только из Европы и Японии. Признак к удалению.

In [94]:
test = test.drop(['vendor'],axis=1) 

### Приведение в соответствие признака Владельцы

In [95]:
train['Владельцы'].value_counts()

In [96]:
test['Владельцы'].value_counts()

In [97]:
train['Владельцы'] = train['Владельцы'].astype('int')

In [98]:
test['Владельцы'] = test['Владельцы'].apply(lambda x: int(re.split(r'[а-я]', x)[0].rstrip()))  ## извлечение числовой величины

In [99]:
train.rename(columns={'Владельцы' : 'owners' }, inplace=True)
test.rename(columns={'Владельцы' : 'owners' }, inplace=True)

### Приведение в соответствие признака Владение

In [100]:
train['Владение'] = train['Владение'].map(time_ownership)

In [101]:
test['Владение'] = test['Владение'].map(time_ownership_test)

In [102]:
train.rename(columns={'Владение' : 'time_ownership' }, inplace=True)
test.rename(columns={'Владение' : 'time_ownership' }, inplace=True)

### Приведение в соответствие признака ПТС

In [103]:
train['ПТС'].value_counts()

In [104]:
test['ПТС'].value_counts()

In [105]:
pts_dict = {'ORIGINAL': 'Оригинал', 'DUPLICATE': 'Дубликат'}

In [106]:
train['ПТС'] = train['ПТС'].replace(pts_dict)

In [107]:
train.rename(columns={'ПТС' : 'pts' }, inplace=True)
test.rename(columns={'ПТС' : 'pts' }, inplace=True)

### Приведение в соответствие признака Привод

In [108]:
train['Привод'].value_counts()

In [109]:
test['Привод'].value_counts()

In [110]:
train.rename(columns={'Привод' : 'drive' }, inplace=True)
test.rename(columns={'Привод' : 'drive' }, inplace=True)

### Приведение в соответствие признака Руль

In [111]:
train['Руль'].value_counts()

In [112]:
test['Руль'].value_counts()

In [113]:
wheel_dict = {'LEFT': 'Левый', 'RIGHT': 'Правый'}

In [114]:
train['Руль'] = train['Руль'].replace(wheel_dict)

In [115]:
train.rename(columns={'Руль' : 'wheel' }, inplace=True)
test.rename(columns={'Руль' : 'wheel' }, inplace=True)

### Приведение в соответствие признака Состояние

In [116]:
train['Состояние'].value_counts()

In [117]:
test['Состояние'].value_counts()

В обучающей выборке информация о признаке отсутствует. Показатель к удалению. 

In [118]:
train = train.drop(['Состояние'],axis=1) 

In [119]:
test = test.drop(['Состояние'],axis=1) 

### Учёт повышения стоимости автомобиля

Согалсно аналитическим отчётам [Тинькофф](https://journal.tinkoff.ru/news/2022-car-cost/), [Автоньюс](https://www.autonews.ru/news/620605ea9a79473bafb9020f) ежемесячно автомобиль на московском вторичном рынке повышается  в цене на 4-6%.  
Сентябрь - октябрь: 4-5%
Ноябрь - 6%
Январь - 5,2%  
С учётом того, что train содержит в себе данные начала сентября, то цены стоит проиндексировать за 5 месяцев 5% ежемесячно.

In [120]:
train['price'] = train['price'].apply(lambda x: x*1.25)

### Приведение в соответствие признака Таможня

In [121]:
train['Таможня'].value_counts()

In [122]:
test['Таможня'].value_counts()

На данной выборке признак не информативен, подлежит удалению

In [123]:
train = train.drop(['Таможня'],axis=1) 
test = test.drop(['Таможня'],axis=1) 

### Приведение в соответствие выборок

In [124]:
set(test.columns).symmetric_difference(train.columns)

Все признаки (кроме цены) подлежат удалению, так как их нельзя восстановить во всех выборках из других признаков

In [125]:
train = train.drop(['start_date', 'hidden'],axis=1) 
test = test.drop(['car_url'],axis=1) 

In [126]:
train = train[['body_type', 'brand', 'color', 'complectation_dict', 'description',
       'engine_displacement', 'engine_power', 'fuel_type', 'mileage',
       'model_date', 'model', 'number_of_doors', 'production_date', 'sell_id',
       'vehicle_transmission', 'owners', 'time_ownership', 'pts', 'drive',
       'wheel', 'price']]

In [127]:
train.dropna(subset=['price'], inplace=True)

In [128]:
pandas_profiling.ProfileReport(train)

In [129]:
pandas_profiling.ProfileReport(test)

Дублей нет. Необходимо заполнить пропуски, удалить дублирующие признаки. Переменнные нуждаются в нормализации и стандартизации. 

In [130]:
# Для анализа склеиваем все датафреймы по общим колонкам, добавляем признак train
train['sample'] = 1  # помечаем где у нас трейн
test['sample'] = 0  # помечаем где у нас тест
test['price'] = 0 # в тесте у нас нет значения цены, мы его должны предсказать, поэтому пока просто заполняем нулями

In [131]:
data = test.append(train, sort=False).reset_index(drop=True) # объединяем

In [132]:
data.info()

## Работа с пропусками

In [133]:
fig, ax = plt.subplots(figsize=(20,12))
sns_heatmap = sns.heatmap(data.isnull(), yticklabels=False, cbar=False, cmap='viridis')

In [134]:
data.isna().sum(axis=0) * 100 / data.shape[0] 

In [135]:
#data[data['complectation_dict'].notnull()].groupby(['brand', 'model', 'model_date'])['body_type'].count()  ## авто с комплектацей
#data[data['complectation_dict'].isnull()].groupby(['brand', 'model', 'model_date'])['body_type'].count()  ## авто без комплектации

82% авто не имеют описания комплектации. Возможно заполнить комплектацию на осноавнии бренда/модели/года выпуска, однако  
не все представленные бренды имеют в сети комплектацию.  
Принято решению удалить столбец  
Также много пропусков по признаку time_ownership, поэтому также удалить этот признак

In [136]:
data = data.drop(['complectation_dict', 'time_ownership'],axis=1)

Пропуск с описанием заполняем формулировкой "Нет информации"

In [137]:
data['description'] = data['description'].fillna('Нет информации')

Признак engine_displacement отсутствует у электромобилей, поэтому решено присовить признаку нулевое значение

In [138]:
data['engine_displacement'] = data['engine_displacement'].fillna(0)

У pts пропуски решено заменить модальным значением

In [139]:
data['pts'] = data['pts'].fillna(data['pts'].mode()[0])

## Анализ непрерывных переменных

In [140]:
# категориальные переменные
cat_cols = ['body_type', 'brand', 'color','fuel_type',
            'model_date', 'model', 'number_of_doors',
            'production_date', 'vehicle_transmission',
            'owners', 'pts', 'drive', 'wheel']
# числовые переменные
num_cols = ['engine_displacement', 'engine_power', 'mileage']
# целевая переменная
target = ['price']

In [141]:
train['price'].hist()

Данные таргета не сбалансированы и нуждаются в стандатизации.

Изображение таргета после логарифмирования

In [142]:
fig = plt.figure(figsize=[8, 8])
ax = fig.add_subplot()
sns.histplot(np.log(data['price']), kde=False)
ax.set_xlabel(xlabel='price', fontdict={'fontsize': 20})
plt.show()

Изображение таргета после нормализации Бокса-Кокса

In [143]:
fig = plt.figure(figsize=[8, 8])
ax = fig.add_subplot()
sns.histplot(stats.boxcox(data['price'][data['price'] > 0].dropna())[0], kde=False)
ax.set_xlabel(xlabel='price', fontdict={'fontsize': 20})
plt.show()

Для целевой перемнной лучше подходит логарифмирование

In [144]:
fig = plt.figure(figsize=[30, 8])
count = 1
for each in num_cols:
    ax = fig.add_subplot(1, 3, count)
    sns.histplot(data[each], kde=False)
    ax.set_xlabel(xlabel=each, fontdict={'fontsize': 20})
    count += 1
plt.show()

Числовые переменные нуждаются в нормализации, так как имеют ассиметрию

Изображение непрерывных переменных после логарифмирования

In [145]:
fig = plt.figure(figsize=[30, 8])
count = 1
for each in num_cols:
    ax = fig.add_subplot(1, 3, count)
    sns.histplot(np.log(data[each]+1), kde=False)
    ax.set_xlabel(xlabel=each, fontdict={'fontsize': 20})
    count += 1
plt.show()

Изображение непрерывных переменных после нормализации Бокса-Кокса

In [146]:
fig = plt.figure(figsize=[30, 8])
count = 1
for each in num_cols:
    ax = fig.add_subplot(1, 3, count)
    sns.histplot(stats.boxcox(
        data[each][data[each] > 0].dropna())[0], kde=False)
    ax.set_xlabel(xlabel=each, fontdict={'fontsize': 20})
    count += 1
plt.show()

Для engine_displacement и engine_power лучше подходит логарифмирование. в то время как для mileage адекватнее преобразование Бокса-Кокса

### Выбросы

In [147]:
boxplot_out(data, num_cols)

In [148]:
graph_out(data, num_cols)

In [149]:
outliers(data, num_cols, 'search')

Выбросов не более 10% процентов, поэтому можно либо оставить, либо сгладить их

### Корреляция

In [150]:
correlation = data[num_cols].corr()
fig, ax = plt.subplots(figsize=(10, 5))
matrix = np.triu(correlation)
sns.heatmap(correlation, annot=True, cmap="BrBG",
            fmt='.2f', linewidths=.5, ax=ax, mask=matrix)

engine_displacement сильно коррелирует с engine_power. Один из признаков подлежит удалению

In [151]:
correlation = data[num_cols + target].corr()
plt.figure(figsize=(3, 5))
sns.heatmap(correlation[['price']].sort_values(
    by='price', ascending=False), vmin=-1, vmax=1, annot=True, cmap='BrBG')

Влияние числовых признаков на целевую переменную незначительное.  
Более адекватно оценить значение переменной в дисперсионном анализе

## Анализ категориальных переменных

In [152]:
hist_type(data, cat_cols)

In [153]:
for each in cat_cols:
    print(each)
    print(data[each].value_counts().head(5))

### Выбросы

In [154]:
def outliers_cat_cols():
    # категориальные переменные с большим количеством вариант
    cat_cols_large = ['body_type', 'brand', 'color',
                      'model_date', 'model', 'production_date']
    
    # категориальные переменные с небольшим количеством вариант
    cat_cols_small = ['fuel_type', 'number_of_doors',
                      'vehicle_transmission','owners',
                      'pts', 'drive', 'wheel']
    
    for each in cat_cols:
        # построение боксплотов для просмотра выбросов
        if each in cat_cols_large:
            fig, ax = plt.subplots(figsize=(40, 10))
        if each in cat_cols_small:
            fig, ax = plt.subplots(figsize=(10, 5))
        sns.boxplot(x=each, y='price', data=data, ax=ax)
        ax.set_title(f'Boxplot for {each}')
        plt.show()
            
        # построение боксплотов для возможности группировки
        if each in cat_cols_large:
            fig, ax = plt.subplots(figsize=(40, 10))
        if each in cat_cols_small:
            fig, ax = plt.subplots(figsize=(10, 5))
        sns.boxplot(x=each, y='price', data=data, ax=ax)
        ax.set_title(f'Boxplot for {each}')
        ax.set_ybound(lower = 0, upper=3*(10**6))
        plt.show()

In [155]:
outliers_cat_cols()

Выбросы есть у большинства типов авто. Некоторые бренды схожи. Например:  
**универсал,седан и фургон  
хэтчбек,компактвен и микровен,    
кабриолет и купэ  
минивен и купе-хардкоп**  
Данные типы можно объединить.

Выбросы есть у большинства брендов. Некоторые бренды схожи. Например:  
**honda, nissan, volvo  
bmw, infiniti,    
skoda, mitsubishi  
chevrolet, citroen, ford  
hyundai, subaru, kia**  
Данные бренды можно объединить.

Выбросы есть у большинства цветов. Некоторые цвета схожи. Например:  
**черный и белый  
красный, голубой, золотистый, пурпурный и серебристый,    
фиолетовый и синий  
chevrolet, citroen, ford**  
Данные цвета можно объединить.

Выбросы есть у большинства видов топлива.  
Виды топлива не подлежат группировке

Выбросы есть по большинству спектру данных признака model_date, особено по авто последних 50 лет.  
Анализировать данные на предмет объединения нет смысла без разреза бренда и модели.

Выбросы есть по большинству спектру данных признака model.  
Анализировать данные на предмет объединения нет смысла без разреза бренда и модели.

Выбросы есть по большинству спектру данных признака production_date, особенно за последние 50 лет.  
Анализировать данные на предмет объединения нет смысла без разреза бренда и модели.

Выбросы есть по всему спектру данных number_of_doors, vehicle_transmission, owners, pts, drive, wheel.  
Данные внутри признаков не подлежит группировке

In [156]:
pandas_profiling.ProfileReport(data)

### Корреляция

Для определения высокой корреляции между категориальными признаками обратимся к корреляции Спирмена.  
production_date и model_date дублируются, поэтому решено удалить production_date

Предложения по созданию новых признаков:  
**model_date  - создание признака моделей до 1973 года включительно   
brand - выделить топовые и редкие бренды  
body_type - выделить топовые и редкие виды авто  
color - выделить топовые и редкие цвета  
создание полиномиальных признаков
создание признаков на основании комментария продавца  
выделение признаков из иинформации о комплектации**

## Корректировка числовых признаков

### Логарифмирование целевой переменной


In [157]:
for each in ['price', 'engine_displacement']:
    data[each] = data[each].apply(lambda x: np.log(x) if x !=0 else x)

### Преобразование Бокса-Кокса

In [158]:
data['mileage'] = stats.boxcox(data['mileage'])[0]

## Удаление шума

In [159]:
outliers(data, num_cols, 'delete')

### результат замены шума на медианное значение  
(замена на модальное значение имело меньший эффект)

In [160]:
outliers(data, num_cols, 'search')

In [161]:
graph_out(data, num_cols + target)

## Создание новых признаков

In [162]:
data['model_old'] = data['model_date'].apply(lambda x: 1 if x <= 1973 else 0)  # 0 - модели после 1973, 1 - модели до 1973 года

In [163]:
body_type_top3 = set(data['body_type'].value_counts().head(3).index)
body_type_top8 = set(data['body_type'].value_counts().head(8).index)
body_type_rare = set(data['body_type'].value_counts().tail(5).index)

In [164]:
brand_top5 = set(data['brand'].value_counts().head(5).index)
brand_top10 = set(data['brand'].value_counts().head(10).index)
brand_rare = set(data['brand'].value_counts().tail(5).index)

In [165]:
color_best = set(data['color'].value_counts().head(1).index)
color_top5 = set(data['color'].value_counts().head(5).index)
color_rare = set(data['color'].value_counts().tail(5).index)

In [166]:
data['body_type_top3'] = data['body_type'].transform(
    lambda x: bool(set(x)&body_type_top3)).astype(int)

data['body_type_top8'] = data['body_type'].transform(
    lambda x: bool(set(x)&body_type_top8)).astype(int)

data['body_type_rare'] = data['body_type'].transform(
    lambda x: bool(set(x)&body_type_rare)).astype(int)

data['brand_top5'] = data['brand'].transform(
    lambda x: bool(set(x)&brand_top5)).astype(int)

data['brand_top10'] = data['brand'].transform(
    lambda x: bool(set(x)&brand_top10)).astype(int)

data['brand_rare'] = data['brand'].transform(
    lambda x: bool(set(x)&brand_rare)).astype(int)

data['color_best'] = data['color'].transform(
    lambda x: bool(set(x)&color_best)).astype(int)

data['color_top5'] = data['color'].transform(
    lambda x: bool(set(x)&color_top5)).astype(int)

data['color_rare'] = data['color'].transform(
    lambda x: bool(set(x)&color_rare)).astype(int)


In [167]:
bin_cols = ['body_type_top3', 'body_type_top8','body_type_rare',
            'brand_top5', 'brand_top10', 'brand_rare',
            'color_best', 'color_top5', 'color_rare']

In [168]:
poly = PolynomialFeatures(2)
data['eng_disp^2'] = poly.fit_transform(data[['engine_displacement', 'mileage']])[:,3]
data['eng_disp*mil'] = poly.fit_transform(data[['engine_displacement', 'mileage']])[:,4]
data['mil^2'] = poly.fit_transform(data[['engine_displacement', 'mileage']])[:,5]

In [169]:
data['eng_disp+mil'] = data[['engine_displacement','mileage']].sum(axis=1)

In [170]:
num_cols_poly = ['engine_displacement', 'engine_power', 'mileage',
                 'eng_disp^2', 'eng_disp*mil', 'mil^2', 'eng_disp+mil']

In [171]:
data.info()

## Label_encoding

In [172]:
# функция приведения переменных типа 'object' к 'int'
def label_encoding(df, cols):
    le = preprocessing.LabelEncoder()
    for each in cols:
        df[each] = df[each].sort_values(ascending=True)
        df[each] = le.fit_transform(df[each].values)
        le_dict = dict(zip(le.classes_, le.transform(le.classes_)))
        print(f'Словарь для параметра {each}:', le_dict)

In [173]:
# категориальные переменные для LabelEncoder
cat_cols_enc = ['body_type', 'brand', 'color','fuel_type',
            'model','vehicle_transmission',
            'pts', 'drive', 'wheel']

In [174]:
label_encoding(data, cat_cols_enc)

### Выбор признаков

In [175]:
data['price'] = data['price'].astype(int)

In [176]:
# Разбиение данных не обучающую и тестовую выборку
train_data = data.query('sample == 1').drop(['sample', 'description'], axis=1)
test_data = data.query('sample == 0').drop(['sample', 'description'], axis=1)

In [177]:
imp_num = pd.Series(f_classif(train_data[num_cols_poly],train_data[target])[0],
                    index=num_cols_poly)
imp_num.sort_values(inplace=True)
imp_num.plot(kind='barh')

In [178]:
cat_cols_new = ['body_type', 'brand', 'color','fuel_type',
                'model','vehicle_transmission',
                'pts', 'drive', 'wheel','model_date',
                'number_of_doors','production_date','owners']

In [179]:
imp_cat = pd.Series(mutual_info_classif(train_data[bin_cols + cat_cols_new], train_data[target]), index=bin_cols + cat_cols_new)
imp_cat.sort_values(inplace=True)
imp_cat.plot(kind='barh', figsize=(30, 10))

In [180]:
correlation = data[num_cols_poly].corr()
fig, ax = plt.subplots(figsize=(10, 5))
matrix = np.triu(correlation)
sns.heatmap(correlation, annot=True, cmap="BrBG",
            fmt='.2f', linewidths=.5, ax=ax, mask=matrix)

Генерация числовых признаков не увеличила эффективности, а только продублировала существующие переменные.  
Из числовых переменных решено включить в модель 'engine_displacement', 'mileage'.  
Из категориальных - 'body_type', 'brand', 'color','fuel_type','model',  
'vehicle_transmission','pts', 'drive', 'wheel','model_date',  
'number_of_doors','owners', 'body_type_top8',  
'body_type_rare', 'brand_top10', 'brand_rare',  
'color_best'

## Переменные для моделирования

In [181]:
# категориальные переменные
cat_cols_mod = ['body_type', 'brand', 'color','fuel_type',
                'model','vehicle_transmission',
                'pts', 'drive', 'wheel','model_date',
                'number_of_doors','owners', 'body_type_top8',
                'body_type_rare', 'brand_top10', 'brand_rare',
                'color_best']
# числовые переменные
num_cols_mod = ['engine_displacement', 'mileage']

In [182]:
y = data.query('sample == 1')['price']

In [183]:
X = data.query('sample == 1')[cat_cols_mod + num_cols_mod]
X_sub = data.query('sample == 0')[cat_cols_mod + num_cols_mod]

## Train Split

In [184]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [185]:
sscale = StandardScaler()
X_num_tr = pd.DataFrame(sscale.fit_transform(X_train[num_cols_mod].values))
X_num_v = pd.DataFrame(sscale.transform(X_val[num_cols_mod].values))
X_num_sub = pd.DataFrame(sscale.transform(X_sub[num_cols_mod].values))


X_cat_tr = X_train[cat_cols_mod].values
X_cat_v = X_val[cat_cols_mod].values
X_cat_sub = X_sub[cat_cols_mod].values

X_train = pd.DataFrame(np.hstack([X_num_tr, X_cat_tr]))
X_val = pd.DataFrame(np.hstack([X_num_v, X_cat_v]))
X_test = pd.DataFrame(np.hstack([X_num_sub, X_cat_sub]))

In [186]:
X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, 

# Model 1: Создадим "наивную" модель 




In [186]:
#tmp_train = X_train.copy()
#tmp_train['price'] = y_train

In [187]:
# Находим median по экземплярам mileage в трейне и размечаем тест
#predict = X_test['brand'].map(tmp_train.groupby('brand')['price'].median())

#оцениваем точность
#print(f"Точность наивной модели по метрике MAPE: {(mape(y_test, predict.values))*100:0.2f}%")

Точность модели по метрике MAPE: 4.49%

# Model 2 : CatBoost

## Fit

In [188]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, y_train,
         #cat_features=cat_features_ids,
         eval_set=(X_val, y_val),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_baseline.model')

In [189]:
# оцениваем точность
predict = model.predict(X_val)
print(f"Точность модели по метрике MAPE: {(mape(y_val, predict))*100:0.2f}%")

Точность модели по метрике MAPE: 1.85%

### Log Traget

In [190]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, np.log(y_train),
         #cat_features=cat_features_ids,
         eval_set=(X_val, np.log(y_val)),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_2_baseline.model')

In [191]:
predict_test = np.exp(model.predict(X_val))

In [192]:
print(f"Точность модели по метрике MAPE: {(mape(y_val, predict_test))*100:0.2f}%")

Точность модели по метрике MAPE: 1.85%, по лидерборду - 89%

# Model 3 : RandomForestRegressor

In [193]:
rf = RandomForestRegressor(random_state = RANDOM_SEED)
rf.fit(X_train, y_train)
y_predict = rf.predict(X_val)
print(f"Точность модели по метрике MAPE: {(mape(y_val, y_predict))*100:0.2f}%")

Точность модели по метрике MAPE: 1.57%, по лидерборду - 27,93%

In [194]:
rf.get_params()

In [195]:
y_predict_rf_sub = rf.predict(X_test)

## Перебор гиперпараметров

In [196]:
random_grid = {'n_estimators': [int(x) for x in np.linspace(start=200, stop=400, num=10)],
               'max_features': ['auto', 'sqrt', 'log2'],
               'max_depth': [int(x) for x in np.linspace(10, 20, num=2)] + [None],
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 4],
               'bootstrap': [True, False]}

rfr = RandomForestRegressor(random_state=RANDOM_SEED)
rf_random = RandomizedSearchCV(estimator=rfr, param_distributions=random_grid,
                               n_iter=100, cv=3, verbose=10, random_state=RANDOM_SEED, n_jobs=-1)

rf_random.fit(X_train, y_train)
rf_random.best_params_

{'n_estimators': 377,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': None,
 'bootstrap': False}

In [197]:
rf_random.best_score_

In [198]:
rf_search = RandomForestRegressor(random_state = RANDOM_SEED,
                                n_estimators =377,
                                min_samples_split=10,
                                min_samples_leaf=1,
                                max_features='sqrt',
                                max_depth=None,
                                bootstrap=False)
rf_search.fit(X_train, y_train)
y_predict = rf_search.predict(X_val)
print(f"Точность модели по метрике MAPE: {(mape(y_val, y_predict))*100:0.2f}%")

Точность модели по метрике MAPE: 1.64%, по лидерборду 28,44%

In [199]:
rf_search.get_params()

In [200]:
y_predict_rf_search_sub = rf_search.predict(X_test)

## Подбор гиперпараметров с помощью Optuna  

In [201]:
def objective(trial):
    
    n_estimators = trial.suggest_int('n_estimators', 0, 400)
    max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2'])
    max_depth = trial.suggest_int('max_depth', 1, 20)
    min_samples_split = trial.suggest_categorical('min_samples_split', [2, 5, 10])
    min_samples_leaf = trial.suggest_categorical('min_samples_leaf', [1, 2, 4])
    bootstrap = trial.suggest_categorical('bootstrap', [True, False])

    
    rfr_random = RandomForestRegressor(random_state=RANDOM_SEED,
                                       n_estimators=n_estimators,
                                       max_features=max_features,
                                       max_depth=max_depth,
                                       min_samples_split=min_samples_split,
                                       min_samples_leaf=min_samples_leaf,
                                       bootstrap=bootstrap
                                      )

    rfr_random.fit(X_train, y_train)

    return rfr_random.score(X_val, y_val)

In [202]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

In [203]:
best_params_rf = study.best_params

In [205]:
best_params_rf

{'n_estimators': 400,
 'max_features': 'sqrt',
 'max_depth': 19,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'bootstrap': False}

In [206]:
rf_optuna = RandomForestRegressor(random_state = RANDOM_SEED,**best_params_rf)
rf_optuna.fit(X_train, y_train)
y_predict = rf_optuna.predict(X_val)
print(f"Точность модели по метрике MAPE: {(mape(y_val, y_predict))*100:0.2f}%")

Точность модели по метрике MAPE: 1.67%

In [208]:
rf_optuna.get_params()

Прошлый запуск Optuna дал лучший результат. MAPE = 1.56%

In [207]:
rf_optuna_old = RandomForestRegressor(random_state = RANDOM_SEED,
                                      n_estimators=374,
                                      max_features='auto',
                                      max_depth=17,
                                      min_samples_split=10,
                                      min_samples_leaf=1,
                                      bootstrap=True)
rf_optuna_old.fit(X_train, y_train)
y_predict = rf_optuna_old.predict(X_val)
print(f"Точность модели по метрике MAPE: {(mape(y_val, y_predict))*100:0.2f}%")

Точность модели по метрике MAPE: 1.56%, по лидерборду - 28,21%

In [209]:
y_predict_rf_opt_sub = rf_optuna_old.predict(X_test)

# Модель 4: ExtraTreesRegressor

In [211]:
etr = ExtraTreesRegressor(random_state = RANDOM_SEED, n_jobs=-1)
etr.fit(X_train, y_train)
y_predict = etr.predict(X_val)
print(f"Точность модели по метрике MAPE: {(mape(y_val, y_predict))*100:0.2f}%")

Точность модели по метрике MAPE: 1.55%. по лидерборду - 28,59%

In [212]:
etr.get_params()

In [213]:
y_predict_etr_sub = etr.predict(X_test)

## Подбор гиперпараметров

In [214]:
random_grid_etr = {'n_estimators': [int(x) for x in np.linspace(start=100, stop=400, num=10)],
                   'max_features': ['auto', 'sqrt', 'log2'],
                   'max_depth': [int(x) for x in np.linspace(5, 20, num=4)] + [None],
                   'min_samples_split': [2, 5, 10],
                   'min_samples_leaf': [1, 2, 4],
                   'bootstrap': [True, False]}

etrr = ExtraTreesRegressor(random_state = RANDOM_SEED, n_jobs=-1)
etr_random = RandomizedSearchCV(estimator=etrr, param_distributions=random_grid_etr,
                                n_iter=100, cv=3, verbose=10, random_state=RANDOM_SEED, n_jobs=-1)

etr_random.fit(X_train, y_train)
etr_random.best_params_

{'n_estimators': 300,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': None,
 'bootstrap': False}

In [215]:
etr_search = ExtraTreesRegressor(random_state = RANDOM_SEED, n_jobs=-1,
                                 n_estimators=300,
                                 min_samples_split=10,
                                 min_samples_leaf=1,
                                 max_features='auto',
                                 max_depth=None,
                                 bootstrap=False)
etr_search.fit(X_train, y_train)
y_predict = etr_search.predict(X_val)
print(f"Точность модели по метрике MAPE: {(mape(y_val, y_predict))*100:0.2f}%")

Точность модели по метрике MAPE: 1.58%, по лидерборду - 28,59%

In [216]:
y_predict_etr_search_sub = etr.predict(X_test)

## Подбор гиперпараметров с помощью Optuna  

In [218]:
def objective_etr(trial):
    
    n_estimators = trial.suggest_int('n_estimators', 1, 400)
    max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2'])
    max_depth = trial.suggest_int('max_depth', 1, 20)
    min_samples_split = trial.suggest_categorical('min_samples_split', [2, 5, 10])
    min_samples_leaf = trial.suggest_categorical('min_samples_leaf', [1, 2, 4])
    bootstrap = trial.suggest_categorical('bootstrap', [True, False])

    
    etr_random = ExtraTreesRegressor(random_state=RANDOM_SEED,
                                     n_estimators=n_estimators,
                                     max_features=max_features,
                                     max_depth=max_depth,
                                     min_samples_split=min_samples_split,
                                     min_samples_leaf=min_samples_leaf,
                                     bootstrap=bootstrap
                                    )

    etr_random.fit(X_train, y_train)

    return etr_random.score(X_val, y_val)

In [219]:
study = optuna.create_study(direction="maximize")
study.optimize(objective_etr, n_trials=100)

In [220]:
best_params_etr = study.best_params

In [221]:
best_params_etr

{'n_estimators': 263,
 'max_features': 'auto',
 'max_depth': 19,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'bootstrap': False}

In [222]:
etr_opt = ExtraTreesRegressor(random_state = RANDOM_SEED, n_jobs=-1, **best_params_etr)
etr_opt.fit(X_train, y_train)
y_predict = etr_opt.predict(X_val)
print(f"Точность модели по метрике MAPE: {(mape(y_val, y_predict))*100:0.2f}%")

Точность модели по метрике MAPE: 1.60%, по лидерборду - 28,21%

In [223]:
y_predict_etr_opt_sub = etr_opt.predict(X_test)

# Модель 5: AdaBoostRegressor

In [228]:
adb = AdaBoostRegressor(random_state=RANDOM_SEED,
                        base_estimator = RandomForestRegressor(random_state = RANDOM_SEED),
                        n_estimators=200, learning_rate=0.5)
adb.fit(X_train, y_train)
y_predict = adb.predict(X_val)
print(f"Точность модели по метрике MAPE: {(mape(y_val, y_predict))*100:0.2f}%")

Точность модели по метрике MAPE: 2.52%, по лидерборду - 29,59%

In [229]:
adb.get_params()

In [230]:
y_predict_adb_sub = adb.predict(X_test)

Подбор гиперпараметров производит сбой ноутбука в работе

# Модель 6: GradientBoostingRegressor

In [193]:
gb = GradientBoostingRegressor(random_state=RANDOM_SEED, max_depth=20, n_estimators=2000)
gb.fit(X_train, y_train)
y_predict = gb.predict(X_val)
print(f"Точность модели по метрике MAPE: {(mape(y_val, y_predict))*100:0.2f}%")

Точность модели по метрике MAPE: 1.60%. по лидерборду - 29,89%

In [194]:
gb.get_params()

In [195]:
y_predict_gb_sub = gb.predict(X_test)

# Подбор гиперпараметров

In [196]:
random_grid_gb = {'n_estimators': [int(x) for x in np.linspace(start=100, stop=400, num=10)],
                  'max_features': ['auto', 'sqrt', 'log2'],
                  'max_depth': [int(x) for x in np.linspace(5, 20, num=4)] + [None],
                  'min_samples_split': [2, 5, 10],
                  'min_samples_leaf': [1, 2, 4],
                  'learning_rate': [x for x in np.linspace(0, 1, num=40)],
                  'loss' : ['squared_error', 'absolute_error', 'quantile']}


gbr = GradientBoostingRegressor(random_state = RANDOM_SEED)
gbr_random = RandomizedSearchCV(estimator=gbr, param_distributions=random_grid_gb,
                                n_iter=100, cv=3, verbose=10, random_state=RANDOM_SEED, n_jobs=-1)

gbr_random.fit(X_train, y_train)
gbr_random.best_params_

In [197]:
gb_search = GradientBoostingRegressor(random_state=RANDOM_SEED,
                                      n_estimators=200,
                                      min_samples_split=2,
                                      min_samples_leaf=1,
                                      max_features='sqrt',
                                      max_depth=None,
                                      loss='quantile',
                                      learning_rate=0.07692307692307693)
gb_search.fit(X_train, y_train)
y_predict = gb_search.predict(X_val)
print(f"Точность модели по метрике MAPE: {(mape(y_val, y_predict))*100:0.2f}%")

Точность модели по метрике MAPE: 1.71%, по лидерборду - 28,62%

In [198]:
gb_search.get_params()

In [199]:
y_predict_gb_search_sub = gb_search.predict(X_test)

## Подбор гиперпараметров с помощью Optuna  

In [205]:
def objective_gb(trial):
    
    n_estimators = trial.suggest_int('n_estimators', 1, 400)
    learning_rate = trial.suggest_float('learning_rate', 0, 1)
    max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2'])
    max_depth = trial.suggest_int('max_depth', 1, 20)
    min_samples_split = trial.suggest_categorical('min_samples_split', [2, 5, 10])
    min_samples_leaf = trial.suggest_categorical('min_samples_leaf', [1, 2, 4])
    
    gb_opt = GradientBoostingRegressor(random_state=RANDOM_SEED,
                                       n_estimators=n_estimators,
                                       learning_rate=learning_rate,
                                       max_features = max_features,
                                       max_depth = max_depth,
                                       min_samples_split = min_samples_split,
                                       min_samples_leaf = min_samples_leaf
                                      )

    gb_opt.fit(X_train, y_train)

    return gb_opt.score(X_val, y_val)

In [206]:
study = optuna.create_study(direction="maximize")
study.optimize(objective_gb, n_trials=100)

In [207]:
best_params_gb = study.best_params

In [208]:
best_params_gb

{'n_estimators': 358,
 'learning_rate': 0.018392465010847778,
 'max_features': 'log2',
 'max_depth': 13,
 'min_samples_split': 2,
 'min_samples_leaf': 4}

In [209]:
gb_opt = GradientBoostingRegressor(random_state=RANDOM_SEED,
                                   **best_params_gb)
gb_opt.fit(X_train, y_train)
y_predict = gb_opt.predict(X_val)
print(f"Точность модели по метрике MAPE: {(mape(y_val, y_predict))*100:0.2f}%")

Точность модели по метрике MAPE: 1.66%, по лидерборду 27,94%

In [210]:
y_predict_gb_opt_sub = gb_opt.predict(X_test)

# Модель 7: HistGradientBoostingRegressor

В рамках Kaggle не удалось реализовать  
По реализации в юпитере:  
Точность модели по метрике MAPE: 2.10%

# Модель 8.1: StackingRegressor  
RandomForestRegressor (базовый вариант)  
RandomForestRegressor (optuna)  
ExtraTreesRegressor (optuna)


In [190]:
estimators = (
    ('rfr', RandomForestRegressor(random_state = RANDOM_SEED)),
    ('rfr_opt', RandomForestRegressor(random_state = RANDOM_SEED,
                                      n_estimators=374,
                                      max_features='auto',
                                      max_depth=17,
                                      min_samples_split=10,
                                      min_samples_leaf=1,
                                      bootstrap=True)),
    ('etr_opt', ExtraTreesRegressor(random_state=RANDOM_SEED, n_jobs=-1,
                                    n_estimators=263,
                                    max_features='auto',
                                    max_depth=19,
                                    min_samples_split=10,
                                    min_samples_leaf=1,
                                    bootstrap=False))
)

meta_model = LinearRegression()

streg = StackingRegressor(estimators=estimators, final_estimator=meta_model,
                          n_jobs=-1, cv=5)

streg.fit(X_train,y_train)
y_predict = streg.predict(X_val)

print(f"Точность модели по метрике MAPE: {(mape(y_val, y_predict))*100:0.2f}%")

Точность модели по метрике MAPE: 1.57%, по лидерборду - 27,87%

In [191]:
y_predict_streg_sub = streg.predict(X_test)

# Модель 8.2.: StackingRegressor  
RandomForestRegressor (базовый вариант)  
RandomForestRegressor (optuna)  
ExtraTreesRegressor (optuna)  
GradientBoostingRegressor (optuna)

In [212]:
estimators = (
    ('rfr', RandomForestRegressor(random_state = RANDOM_SEED)),
    ('rfr_opt', RandomForestRegressor(random_state = RANDOM_SEED,
                                      n_estimators=374,
                                      max_features='auto',
                                      max_depth=17,
                                      min_samples_split=10,
                                      min_samples_leaf=1,
                                      bootstrap=True)),
    ('etr_opt', ExtraTreesRegressor(random_state=RANDOM_SEED, n_jobs=-1,
                                    n_estimators=263,
                                    max_features='auto',
                                    max_depth=19,
                                    min_samples_split=10,
                                    min_samples_leaf=1,
                                    bootstrap=False)),
    ('gb_opt', GradientBoostingRegressor(random_state=RANDOM_SEED,
                                         n_estimators=358,
                                         learning_rate=0.018392465010847778,
                                         max_features='log2',
                                         max_depth=13,
                                         min_samples_split=2,
                                         min_samples_leaf=4))
)

meta_model = LinearRegression()

streg_reg = StackingRegressor(estimators=estimators, final_estimator=meta_model,
                          n_jobs=-1, cv=5)

streg_reg.fit(X_train,y_train)
y_predict = streg_reg.predict(X_val)

print(f"Точность модели по метрике MAPE: {(mape(y_val, y_predict))*100:0.2f}%")

Точность модели по метрике MAPE: 1.61%, по лидерборду - 27,94%

In [213]:
y_predict_streg_reg_sub = streg_reg.predict(X_test)

# Модель 8.3: StackingRegressor  
RandomForestRegressor (базовый вариант)  
GradientBoostingRegressor (optuna)

In [215]:
estimators = (
    ('rfr', RandomForestRegressor(random_state = RANDOM_SEED)),
    ('gb_opt', GradientBoostingRegressor(random_state=RANDOM_SEED,
                                         n_estimators=358,
                                         learning_rate=0.018392465010847778,
                                         max_features='log2',
                                         max_depth=13,
                                         min_samples_split=2,
                                         min_samples_leaf=4))
)

meta_model = LinearRegression()

streg_rg = StackingRegressor(estimators=estimators, final_estimator=meta_model,
                          n_jobs=-1, cv=5)

streg_rg.fit(X_train,y_train)
y_predict = streg_rg.predict(X_val)

print(f"Точность модели по метрике MAPE: {(mape(y_val, y_predict))*100:0.2f}%")

Точность модели по метрике MAPE: 1.63%

In [213]:
y_predict_streg_rg_sub = streg_rg.predict(X_test)

# Submission

Лучшими моделями согласно MAPE стали модели:  
**RandomForestRegressor (базовый вариант)** 1,57% / ЛДБ - 27,93%  
**RandomForestRegressor (с перебором гиперпараметров с помощью optuna)** 1,56% / ЛДБ - 28,21%  
**ExtraTreesRegressor (с перебором гиперпараметров с помощью optuna)** 1,60% / ЛДБ - 28,21%  
**GradientBoostingRegressor (с перебором гиперпараметров с помощью optuna)** 1,66% / ЛДБ - 27,94%  

**Стэккинг** этих моделей в разных сочетаниях позволил выделить лучшую модель:  
  
RandomForestRegressor (базовый вариант)  
RandomForestRegressor (optuna)  
ExtraTreesRegressor (optuna)  

где мета - моделью была Линейная регрессия

В итоге MAPE = 1,57 / ЛДБ - 27,87%


In [214]:
sample_submission['price'] = np.round(np.exp(y_predict_streg_sub),0)
sample_submission.to_csv(f'submission_2_v{VERSION}.csv', index=False)
sample_submission.head(10)

## Выполнено

* #### Написан код для парсинга с помощью Selenium и BeautifulSoup  
* #### Приведение обучающей и тестовой выборки к единообразию  
* #### Учтена временная разница между выборками при адаптации цены на авто   
* #### Проанализированы полученные признаки  
* #### Заполнены пропуски  
* #### Проанализирована взаимосвязь признаков  
* #### Рассмотрены разные варианты удаления шума  
* #### Созданы новые параметры  
* #### Выбраны параметры для моделирования  
* #### Нормализованы некоторые признаки  
* #### Построена наивная модель  
* #### Протестирован CatBoost  
* #### Реализованы модели:  
 #### RandomForestRegressor  
 #### ExtraTreesRegressor  
 #### AdaBoostRegressor  
 #### GradientBoostingRegressor  
 #### HistGradientBoostingRegressor   
* #### Для трёх из пяти моделей оптимизированы гиперпараметры с помощью Optuna и SearchCV
* #### Построенные модели оценены с помощью метрики MAPE  
* #### Для лучших моделей выполнен стэкинг  

## Анализ результатов моделирования

* #### Разница между статистикой на валидации и на тесте возможно является причиной переобучения  
* #### Одной из лучших моделей является Случайный лес с параметрами по умолчанию, что возможно объясняется некачественными данными.  
* #### Последнее показывает необходимость дополнительных итераций по EDA и ML

## Проблемы при выполнении проекта

* #### Длительный парсинг (написанный код с помощью selenium затрачивал много времени на функцию get).
* #### В результате парсинга за 3 суток было прочитано лишь 9000 ссылок  
* #### Реализация парсинга в GoogleColab вызыавла ошибки при чтении XPath  
* #### Затраты времени на выполнение операций по подбору гиперпараметров  
* #### Обновление Kaggle после 20 минутного простоя и выполнение кода заново

## Точки роста

* #### Определение границ гиперпараметров при реализации SearchCV  
* #### Обработка текстовых признаков. Возможно это не поможет на данной выборке без дополнительного парсинга, так как много пропусков по текстовым данным.  
* #### Более эффективный Feature Engineering (формирование групп внутри определенных признаков, таких как модели, бренды и т.д.)  
* #### Эффективная по времени реализация парсинга  
* #### Валидация